# Learning Objectives:

- Understand data acquisition: downloading from static links, crawling through entire websites, and streaming data from real-time sources
- Understand data management: organizing data directories

## Where am I?

In [1]:
getwd()

[1] "/home/lngo/git/data-mining-r-workshop"

In [2]:
setwd("/home/lngo/data-mining-r/")

ERROR: Error in setwd("/home/lngo/data-mining-r/"): cannot change working directory


In [ ]:
getwd()

## Data Acquisition

We are looking at a list of academic institutions from the Carnegie Classifications. These academic institutions share the same characteristics

- Graduate Instructional Program = "Research Doctoral: Highest Research Activity" 
- Enrollment Profile = "High undergraduate" 
- Basic = "Doctoral Universities" 
- Level = "4-year or above"

The list of institutions is available at https://raw.githubusercontent.com/clemsonciti/data-mining-r-workshop/master/institutions.txt

### Create data directory

In [ ]:
current_dir <- getwd()
data_dir <- 'data'

if (!file.exists(data_dir)){
    dir.create(file.path(current_dir, data_dir))
} else {
    print ("Directory already exists")
}

### Download data from static link



In [ ]:
institutions <- file.path(current_dir, data_dir, "institutions.csv")
institutions_url <- 'https://raw.githubusercontent.com/clemsonciti/data-mining-r-workshop/master/institutions.txt'

download.file(institutions_url, institutions)

In [ ]:
df_institutions <- read.csv(institutions)

In [ ]:
head(df_institutions)

### Automate the process with multiple static links

Most website's itemized contents (i.e., pages, list of items, etc.) are designed with listing patterns in mind. By identifying and extracting these patterns, it is possible to automate the data acquisition process

** Example 1 **
- https://ncsesdata.nsf.gov/profiles/site?method=view&fice=3963
- https://ncsesdata.nsf.gov/profiles/site?method=view&fice=1081

** Example 2 **
- https://www.yelp.com/biz/emerils-new-orleans-new-orleans
- https://www.yelp.com/biz/emerils-new-orleans-new-orleans?start=20
- https://www.yelp.com/biz/emerils-new-orleans-new-orleans?start=40
- https://www.yelp.com/biz/emerils-new-orleans-new-orleans?start=60
- ....

### Data Management

You are about to download a bunch of files to your workstation. How are you going to organize the new data files?

- Directory and file's names should provide metadata knowledge and serve as an easy way to categorize contents
- Spaces and special characters in names should be avoided at all cost

In [ ]:
# names including data sources: lengthy but meaningful and maintainable
ncses_institution_profiles_dir <- 'ncses_institution_profiles'

if (!file.exists(file.path(data_dir, ncses_institution_profiles_dir))){
    dir.create(file.path(data_dir, ncses_institution_profiles_dir))
    print ('Create directory ncses_institution_profiles under data')
} else {
    print ("Directory already exists")
}

In [ ]:
url_prefix <- 'https://ncsesdata.nsf.gov/profiles/site?method=view&fice='
for (i in 1:nrow(df_institutions)){
    full_url <- paste(url_prefix, df_institutions$FICE[i],sep='')
    print (full_url)
}

The previous links are just HTML pages. We need to identify the download URL of the data files. For example:
- https://ncsesdata.nsf.gov/profiles/site?method=download&fice=1081

In [ ]:
# how to name your downloaded file? Institution Name or FICE?
# Why not BOTH?
# How to name your files' extension?
for (i in 1:nrow(df_institutions)){    
    institution <- paste(df_institutions$FICE[i],df_institutions$Institutions[i],'.zip',sep='_')
    print(institution)
}

In [ ]:
# Spaces in the file names! Spaces in the file names!
for (i in 1:nrow(df_institutions)){    
    institution <- paste(df_institutions$FICE[i],df_institutions$Institutions[i],'.zip',sep='_')
    institution <- gsub(" ", "_", institution, fixed = TRUE)
    print(institution)
}

*Why do we replace spaces with an underscore?*

In [ ]:
# We are really downloading the files now, and also cleaning up the file names ...
url_prefix <- 'https://ncsesdata.nsf.gov/profiles/site?method=download&fice='
for (i in 1:nrow(df_institutions)){
    full_url <- paste(url_prefix, df_institutions$FICE[i],sep='')
    institution <- paste(df_institutions$FICE[i],df_institutions$Institutions[i],sep='_')
    institution <- gsub(" ", "_", institution, fixed = TRUE)
    institution <- paste(institution,'.zip',sep='')
    institution_path <- file.path(current_dir,data_dir, ncses_institution_profiles_dir, institution)
    print(full_url)
    print(institution_path)
    download.file(full_url, institution_path, mode = "wb")
    # be courteous to your source:
    sleep_time <- sample(2:6,1)
    print (sleep_time)
    Sys.sleep(sleep_time)
}

### Collecting Streaming Data (RSS feeds, Twitter)

- feedeR for RSS feeds: https://cran.r-project.org/web/packages/feedeR/feedeR.pdf
- rtweet for Twitter data: https://cran.r-project.org/web/packages/rtweet/rtweet.pdf

** Streaming RSS feeds **

In [ ]:
library(feedeR)

In [ ]:
reddit_feed <- feed.extract("https://www.reddit.com/r/all/new/.rss")

In [ ]:
str(reddit_feed)

In [ ]:
str(reddit_feed$items)

In [ ]:
reddit_feed$items

In [ ]:
for (i in 1:10){
    reddit_feed <- feed.extract("https://www.reddit.com/r/all/new/.rss")
    print(reddit_feed$updated)
    print(reddit_feed$items[1]$title[1])
    print(reddit_feed$items[3]$link[1])
}

** Streaming Twitter feeds **

In [ ]:
library(rtweet)

clemson_tweets <- stream_tweets(q = 'Clemson,clemson,ClemsonTiger', timeout = 120)

In [ ]:
str(clemson_tweets)

In [ ]:
clemson_tweets

** Question to be considered: **
- What is the rate of data acquisition?
- Am I loosing data?
- Am I duplicating data?